In [ ]:
Vilohit
2307751

In [1]:
pip install pandas

In [8]:
import pandas as pd
!wget https://norvig.com/ngrams/count_1w.txt
!wget https://norvig.com/ngrams/count_2w.txt

unigrams_df=pd.read_csv('https://norvig.com/ngrams/count_1w.txt')
bigrams_df=pd.read_csv('https://norvig.com/ngrams/count_2w.txt')

totalUnigrams=len(unigrams_df)
totalBigrams=len(bigrams_df)

print(f'Number of unigrams:{totalUnigrams} Number of Bigrams: {totalBigrams}')
display(unigrams_df.head(100),bigrams_df.head(100))

--2023-10-27 12:36:22--  https://norvig.com/ngrams/count_1w.txt
Resolving norvig.com (norvig.com)... 158.106.138.13
Connecting to norvig.com (norvig.com)|158.106.138.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4956241 (4.7M) [text/plain]
Saving to: ‘count_1w.txt.2’

count_1w.txt.2      100%[===================>]   4.73M  2.85MB/s    in 1.7s    

2023-10-27 12:36:25 (2.85 MB/s) - ‘count_1w.txt.2’ saved [4956241/4956241]

--2023-10-27 12:36:25--  https://norvig.com/ngrams/count_2w.txt
Resolving norvig.com (norvig.com)... 158.106.138.13
Connecting to norvig.com (norvig.com)|158.106.138.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5566017 (5.3M) [text/plain]
Saving to: ‘count_2w.txt.2’

count_2w.txt.2      100%[===================>]   5.31M  2.75MB/s    in 1.9s    

2023-10-27 12:36:28 (2.75 MB/s) - ‘count_2w.txt.2’ saved [5566017/5566017]

Number of unigrams:333332 Number of Bigrams: 286357


,the\t23135851162
0,of\t13151942776
1,and\t12997637966
2,to\t12136980858
3,a\t9081174698
4,in\t8469404971
...,...
95,service\t519537222
96,x\t508609523
97,than\t502609275
98,find\t502043038


,0Uplink verified\t523545
0,0km to\t116103
1,1000s of\t939476
2,100s of\t539389
3,100th anniversary\t158621
4,10am to\t376141
...,...
95,25th anniversary\t261023
96,25th of\t397735
97,26th of\t271707
98,27th of\t276619


In [29]:
def probability(sentence):
    words = sentence.split()
    sentenceProbability = 1.0

    for reqWord in words:
        rows = unigrams_df[unigrams_df[unigrams_df.columns[0]] == reqWord].values
        if len(rows) > 0:
            word_count = rows[0][1]
            sentenceProbability *= word_count / totalUnigrams
        else:
            return 0

    for i in range(len(words) - 1):
        bigram = words[i] + " " + words[i + 1]
        bigram_rows = bigrams_df[bigrams_df[bigrams_df.columns[0]] == bigram].values
        unigram_rows = unigrams_df[unigrams_df[unigrams_df.columns[0]] == words[i]].values

        if len(bigram_rows) > 0 and len(unigram_rows) > 0:
            bigram_count = bigram_rows[0][1]
            unigram_count = unigram_rows[0][1]
            sentenceProbability *= bigram_count / unigram_count
        else:
            return 0

    return sentenceProbability


probability("i love you")>probability('i hate you')

False

In [10]:
unigram_counts = {'i': 100, 'love': 50, 'you': 80, 'hate': 20}
bigram_counts = {('i', 'love'): 30, ('love', 'you'): 40, ('i', 'hate'): 10, ('hate', 'you'): 15}

total_unigrams = sum(unigram_counts.values())
unigram_probabilities = {word: count / total_unigrams for word, count in unigram_counts.items()}

total_bigrams = sum(bigram_counts.values())
bigram_probabilities = {bigram: count / total_bigrams for bigram, count in bigram_counts.items()}

In [11]:
def probability(sentence):
    words = sentence.split()
    sentence_probability = 1.0
    for i in range(len(words) - 1):
        bigram = (words[i], words[i + 1])
        if bigram in bigram_probabilities:
            sentence_probability *= bigram_probabilities[bigram]
        else:
            sentence_probability = 0.0
            break
    sentence_probability *= unigram_probabilities.get(words[-1], 0)
    return sentence_probability

In [12]:
probability_love = probability("i love you")
probability_hate = probability("i hate you")

print("Probability of 'i love you':", probability_love)
print("Probability of 'i hate you':", probability_hate)
print("Is 'i love you' more probable than 'i hate you'?", probability_love > probability_hate)

Probability of 'i love you': 0.042548476454293625
Probability of 'i hate you': 0.005318559556786703
Is 'i love you' more probable than 'i hate you'? True


In [28]:
import math
import pandas as pd
bigrams_df = pd.DataFrame({'word1': ['i', 'i', 'love', 'love'], 'word2': ['love', 'hate', 'you', 'him'], 'count': [30, 10, 40, 5]})

def shannon_visualization(bigrams_df, context_word):
    context_bigrams = bigrams_df[bigrams_df['word1'] == context_word]

    total_count = context_bigrams['count'].sum()

    visualization_values = {}
    for _, row in context_bigrams.iterrows():
        word = row['word2']
        count = row['count']
        probability = count / total_count
        visualization_values[word] = -math.log2(probability)

    return visualization_values

context_word = 'i'  # Example context word
visualization_results = shannon_visualization(bigrams_df, context_word)

print("Shannon Visualization values for words following '{}':".format(context_word))
for word, value in visualization_results.items():
    print("{}: {:.2f}".format(word, value))


Shannon Visualization values for words following 'i':
love: 0.42
hate: 2.00


In [24]:
columns = ['words', 'count']
unigrams_df = pd.read_csv('/content/count_1w.txt',names= columns, delimiter='\t')
bigrams_df = pd.read_csv('/content/count_2w.txt',names = columns,delimiter='\t')

In [27]:
unigrams_df[unigrams_df['words']=='i']

,words,count
13,i,3086225277
